## Importing the Required Libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations
import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.applications.inception_resnet_v2 import InceptionResNetV2
#import itertools
#import shutil
%matplotlib inline

## Importing Image Dataset

In [ ]:
cwd_path = os.getcwd()
images_path = os.path.join(cwd_path, 'image_dataset/dataset')

# List the clouds in the dataset folder: 10 Clouds
list_clouds = [name for name in os.listdir(images_path)]
# Was not required for MS OS
#list_clouds.remove('.DS_Store')
list_clouds

In [ ]:
# Create dictionary to store the list of image files based off cloud name
dict_clouds = {}

# Loop thru list of clouds
for cloud in list_clouds:
    # Get files based off each cloud
    cloud_path = os.path.join(images_path, cloud + '/positive')
    files = [os.path.join(cloud_path, f) for f in os.listdir(cloud_path) if os.path.isfile(os.path.join(cloud_path, f))]
    # add cloud and file list into dictionary
    dict_clouds[cloud] = files

len(dict_clouds)

## Image Pre-processing

In [ ]:
def prepare_image(img):
    # Convert the image to a numpy array
    img = image.img_to_array(img)
    # Scale from 0 to 255
    img /= 255
    # Invert the pixels
    img = 1 - img
    # Flatten the image to an array of pixels
    image_array = img.flatten().reshape(-1, 299 * 299)
    # Return the processed feature array
    return image_array

In [ ]:
from keras.preprocessing import image
# Load the saved image using Keras and resize it to the
# format of 299x299 pixels
image_size = (299, 299)
filepath = dict_clouds['cirrus'][0]
im = image.load_img(filepath, target_size=image_size,
                    grayscale=True)

# Convert the 2D image to an array of pixel values
image_array = prepare_image(im)

img = image.load_img(filepath, target_size=image_size)
print(image_array)

In [ ]:
# Load image and resize to the input dimensions that InceptionResnetV2 will use for training
imgplot = plt.imshow(img)

## Split Data for training

In [ ]:
# Split data for training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Configure the Model for Training

In [ ]:
# Model soucre - https://keras.io/applications/#inceptionresnetv2
model = InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=11)

In [ ]:
# Compile model
model.compile(Adam(lr=0.0001), 
              loss=’binary_crossentropy’,
              metrics=[‘accuracy’])

In [ ]:
# Summarize our model
model.summary()

## Model Checkpoint

In [ ]:
filepath = “model.h5”
checkpoint = ModelCheckpoint(filepath, monitor=’val_acc’, 
                        verbose=1, save_best_only=True, mode=’max’)